In [1]:
import pathlib
import os
if not "changed_working_directory" in locals():
    changed_working_directory = True
    os.chdir(pathlib.Path().resolve().parent)
    
%load_ext autoreload
%autoreload 2

In [2]:
from tabGAN import TabGAN, TabGANcf
from src import constants as const
import helpers

imports_path = "src//imports.ipynb"
tabGAN_path = "tabGAN.py"
utils_path = "utils\\utils.ipynb"

%run "$imports_path"

if const.dir.project() != os.getcwd():
    raise ValueError("Project directory not the same as in global vars")

In [11]:
dataset_train_path = os.path.join(const.dir.data(), "df_adult_edited_train.csv")
dataset_test_path = os.path.join(const.dir.data(), "df_adult_edited_test.csv")

data_train = pd.read_csv(dataset_train_path)
data_test = pd.read_csv(dataset_test_path)
discrete_columns = data_train.columns[data_train.dtypes == "object"]

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
# y = 1 * x_0 + 2 * x_1 + 3
y = np.dot(X, np.array([1, 2])) + 3
reg = LinearRegression().fit(X, y)
reg.score(X, y)

1.0

In [92]:
reg.predict([[0,0]])

array([3.])

array([1., 2.])

In [74]:
@tf.custom_gradient
def linear_pred(y):
  def backward(dy):
    return y * reg.coef_.reshape((1, reg.coef_.shape[0]))
  return tf.py_function(reg.predict,
                       inp=[x], Tout=tf.float32), backward

In [77]:
with tf.GradientTape() as tape:
    x = tf.constant([[1., 3.],
                    [2., 1]])
    tape.watch(x)
    print("x:", x)
    y = linear_pred(x)
    print("y:", y)
print(tape.gradient(y, x))

x: tf.Tensor(
[[1. 3.]
 [2. 1.]], shape=(2, 2), dtype=float32)
y: tf.Tensor([10.  7.], shape=(2,), dtype=float32)
tf.Tensor(
[[1. 6.]
 [2. 2.]], shape=(2, 2), dtype=float32)


In [2]:
with tf.GradientTape() as tape:
    x = tf.constant([[1., 3.]])
    tape.watch(x)
    print("x:", x)
    a = 1.5 * tf.py_function(reg.predict,
                       inp=[x], Tout=tf.float32)
    y = a
    print("y:", y)
    tape.watch(y)
print(tape.gradient(y, x))

NameError: name 'tf' is not defined

In [51]:
import sklearn
?sklearn.linear_model.LinearRegression

In [4]:
tgcf = TabGANcf(data_train, lambda x: np.random.uniform(size=x.shape[0]), n_critic=10,
               quantile_transformation_int=True, quantile_rand_transformation=True, noise_discrete_unif_max=0.4)
tgcf.train(10, progress_bar=True)

  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
tgcf.sample()

,age,workclass,fnlwgt,education,educational.num,marital.status,occupation,race,gender,capital.gain,capital.loss,hours.per.week,native.country,income
0,36.000000,Government,39203.953125,Assoc-voc,13.0,Never-married,Exec-managerial,White,Male,0.0,0.0,40.0,US,<=50K
1,35.000000,Private,208880.671875,<=12th,7.0,Never-married,Tech-support,White,Male,0.0,0.0,40.0,US,<=50K
2,28.000000,Private,43067.070312,<=12th,4.0,Married,Other-service,White,Male,0.0,0.0,40.0,US,<=50K
3,55.000000,Private,208543.250000,HS-grad,9.0,Widowed,Machine-op-inspct,White,Male,0.0,0.0,18.0,US,<=50K
4,28.000000,Private,155674.875000,Bachelors,10.0,Separated,Adm-clerical,White,Male,0.0,0.0,48.0,US,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30157,30.000000,Private,551353.562500,Some-college,10.0,Never-married,Handlers-cleaners,White,Female,0.0,0.0,60.0,Non-US,<=50K
30158,46.290089,Private,180144.843750,Bachelors,16.0,Married,Prof-specialty,Asian-Pac-Islander,Male,0.0,0.0,40.0,Non-US,>50K
30159,22.000000,Private,346421.875000,Some-college,9.0,Married,Craft-repair,White,Female,0.0,0.0,30.0,US,>50K
30160,33.000000,Government,119276.476562,Assoc-acdm,13.0,Married,Farming-fishing,White,Female,0.0,0.0,40.0,US,<=50K
